## Imports

In [ ]:
#Import necessary libraries and functions
#!pip install opencv-python
import requests 
from bs4 import BeautifulSoup 
import re
import json
#import urllib
import os
import urllib.request
import cv2
import json
import scipy.misc
import pandas as pd
from undistort_field import *
from pass_detection import *
import numpy as np
from glob import glob
import time

In [ ]:
from skimage.morphology import skeletonize

In [ ]:
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

## Scrape the Charts

In [ ]:
teams = ["arizona-cardinals",
	"atlanta-falcons",
	"baltimore-ravens",
	"buffalo-bills",
	"carolina-panthers",
	"chicago-bears",
	"cincinnati-bengals",
	"cleveland-browns",
	"dallas-cowboys",
	"denver-broncos",
	"detroit-lions",
	"green-bay-packers",
	"houston-texans",
	"indianapolis-colts",
	"jacksonville-jaguars",
	"kansas-city-chiefs",
	"los-angeles-chargers",
	"los-angeles-rams",
	"miami-dolphins",
	"minnesota-vikings",
	"new-england-patriots",
	"new-orleans-saints",
	"new-york-giants",
	"new-york-jets",
	"oakland-raiders",
	"philadelphia-eagles",
	"pittsburgh-steelers",
	"san-francisco-49ers",
	"seattle-seahawks",
	"tampabay-buccaneers",
	"tennessee-titans",
	"washington-redskins"
]

# seasons = ["2017","2018","2019"]
seasons = ['2019']
weeks = ["1", "2", "3", "4", "5",
         "6", "7", "8", "9", "10",
         "11", "12", "13",
         '14', "15", "16", "17",
         "wild-card", "divisional",
         "conference", "super-bowl"]

pattern = re.compile("charts")

print("Scraping images and html data...")

for team in teams:
	for season in seasons:
		print(team, "\t", season)
		for week in weeks:
			URL = "https://nextgenstats.nfl.com/charts/list/route/" + team + "/" + season + "/" + week
			try:
				r = requests.get(URL)
			except:
				print(URL + '  is giving an error. Skipping...')    
				continue
			soup = BeautifulSoup(r.content, "html.parser")

			script = soup.find_all("script", text=pattern)
			if len(script)==0:
				print(URL + ' is giving an error. Skipping...')    
				continue
            
			contains_charts = json.loads(str(script[0])[33:-131])

			if (len(contains_charts["charts"]["charts"]) != 0):
				for chart in contains_charts["charts"]["charts"]['charts']:

					name = chart["lastName"] + "_" + chart["firstName"] + "_" + chart["position"]
					chart["team"] = team

					folder = str("Route_Charts" + os.sep + team + os.sep + season + os.sep + week + os.sep)
					img_folder = folder + "images" + os.sep
					data_folder = folder + "data" + os.sep

					if not os.path.exists(img_folder):
						os.makedirs(img_folder)
					if not os.path.exists(data_folder):
						os.makedirs(data_folder)

					img_file = img_folder + name + ".jpeg"
					url = "https:" + chart["extraLargeImg"]
					try:
						urllib.request.urlretrieve(url, img_file)
					except:
						print(URL + '  is giving an error. Skipping...') 
						continue
# 					urllib.request.urlretrieve(url, img_file)

					data_file = data_folder + name + ".txt"
					with open(data_file, 'w') as datafile: 
						json.dump(chart, datafile)

print("Done.")


## Clean the charts

In [ ]:
#Clean the images from the charts ('clean.py')
def new_image(image):

	img_name = image.split(os.sep)[-1].split(".")[0]
	img = cv2.imread(image)

	if (img.shape[0:2] == (1200, 1200)):
		crop_img = img[0:680, 0:1200]
		temp_name = img_name + "_temp.jpg"
		cv2.imwrite(temp_name, crop_img)
	else:
		print("Image must be of size (1200, 1200)")
		return

	clean_img = clean_field(temp_name)
	write_path = clean_path + os.sep + os.sep.join(image.split(os.sep)[1:-1]) 
	if not os.path.exists(write_path): os.makedirs(write_path)

	if (clean_img != None):
		write_name = write_path + os.sep + img_name + ".jpeg"
		scipy.misc.imsave(write_name, clean_img)
	os.remove(temp_name)

def new_data(folder, image): 
	data_path = os.sep.join(folder.split(os.sep)[:-1]) + os.sep + "data" 
	data_file = data_path + os.sep + image.split(".")[0] + ".txt"
	if 'St.' in data_file:
		return
	new_data_path = clean_path + os.sep + os.sep.join(folder.split(os.sep)[1:-1]) + os.sep + "data" 
	new_data_file = new_data_path + os.sep + image.split(".")[0] + ".txt"

	if not os.path.exists(new_data_path): 
		os.makedirs(new_data_path)

	with open(data_file) as _file: 
		old_data = json.load(_file)
		new_data = {key: old_data[key] for key in keys}
	with open(new_data_file, "w") as _file:
		json.dump(new_data, _file)



if __name__ == '__main__':
	keys = ['receptions', 'receivingYards', 'touchdowns', 'playerNameSlug', 
		'teamId', 'playerName', 'season', 'position',
		'type', 'week', 'gameId', 'esbId', 'firstName', 'lastName', 
		'team'
	]

	clean_path = "Cleaned_Route_Charts"

	if not os.path.exists(clean_path): os.makedirs(clean_path)

	pass_chart_folders = [folder[0] for folder in os.walk("Route_Charts")]
	image_folders = [folder for folder in pass_chart_folders if folder.split(os.sep)[-1] == "images"]
	print("Cleaning images and data...")
	for folder in image_folders:
		#print folder 
		images = os.listdir(folder)
		for image in images:
# 			print(image)
# 			print(folder)
			clean_folder = 'Cleaned_Route_Charts\'' + folder[12:]            
			name = os.path.join(clean_folder,image)
			if os.path.isfile(name):
				continue
			if not image.startswith("."): 
				try:
					new_image(os.path.join(folder, image))
					new_data(folder, image)
				except:
					continue
	print("Done.")


## Route extraction

If anyone has ideas to improve this, please do not hesitate to contribute

In [ ]:
def map_route_locations(image,td):
    lower_green = np.array([40,100, 100])
    upper_green = np.array([80, 255, 255])

    lower_white = np.array([230, 230, 230])
    upper_white = np.array([255, 255, 255])

    lower_gray = np.array([126, 126, 126])
    upper_gray = np.array([132, 132, 132])
    col_names = ["route_type", "x", "y"]
    route_locations = pd.DataFrame(columns = col_names)

    image = cv2.imread(image)
    row, col = image.shape[0:2]
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(hsv, lower_green, upper_green)
    mask2 = cv2.inRange(image, lower_white, upper_white)
    mask3 = cv2.inRange(image, lower_gray, upper_gray)

    # Bitwise-AND mask and original image
    c_pixels = cv2.bitwise_and(image, image, mask=mask2)
    c_pixels = c_pixels.clip(0,1)
    ske_c = skeletonize(c_pixels[:,:,1]).astype(np.uint8)
    
    yac_pixels = cv2.bitwise_and(image, image, mask=mask)
    yac_pixels = cv2.cvtColor(yac_pixels, cv2.COLOR_HSV2BGR).clip(0,1)
    ske_yac = skeletonize(yac_pixels[:,:,2]).astype(np.uint8)
    
    inc_pixels = cv2.bitwise_and(image, image, mask=mask3)
    inc_pixels = inc_pixels.clip(0,1)
    ske_inc = skeletonize(inc_pixels[:,:,1]).astype(np.uint8)
    

    x,y = np.where(ske_c != 0)
    X = np.stack((x,y),axis=1 )
    c_routes = X
   
    x,y = np.where(ske_yac != 0)
    X = np.stack((x,y),axis=1 )
    yac_routes = X

    x,y = np.where(ske_inc != 0)
    X = np.stack((x,y),axis=1 )
    inc_routes = X


    sideline = 40 # pixels
    width = 53.33 # standard width of football field
    center_x = col/2

    if col > 1370:
        _75_yd_line = 0
        LOS = 596

        _1_yd_x = float(col - sideline*2)/width
        _1_yd_y = float(LOS - _75_yd_line)/75

    else:
        _55_yd_line = 5
        LOS = 572
        _1_yd_x = float(col - sideline*2)/width
        _1_yd_y = float(LOS - _55_yd_line)/55

    route_type='COMPLETE'
    y = c_routes[:,0]
    x = c_routes[:,1]
    y_loc = (LOS - y)/_1_yd_y
    x_loc = (x - center_x)/_1_yd_x
    loc = [x_loc,y_loc]
    t_loc = np.transpose(loc)
    
    # TOUCHDOWN REMOVAL CODE
    # - Very very crude method of removing TD annotations from
    # the charts. If anyone has better ideas (computer vision is likely the way to go here),
    #please do not hesitate to reach out
    for t in range(0,td):
        old_minimum=20
        new_points = np.zeros(shape=(102,2))
        final_points = new_points.copy()
        for i in range(0,275):
        #     print(i*.5)
            for j in range(0,275):
                new_points[:,0] = td_points[:,0]-i*.2
                new_points[:,1] = td_points[:,1]+j*.2
                C = cdist(t_loc, new_points)
                minimum = sum(C.min(axis=0))
                if minimum < old_minimum:
                    old_minimum=minimum
                    td_row, td_col = linear_sum_assignment(C)
                    final_points = new_points.copy()
        try:
            td_row

        except:
            print('couldn''t find TD')
            break

        td_rows =list(td_row)
        mask = np.ones(len(t_loc), dtype=bool)
        mask[td_rows] = False
        t_loc = t_loc[mask]
    
    df = pd.DataFrame(t_loc,columns=['x','y'])
    df['route_type'] = route_type
    route_locations = route_locations.append(df, ignore_index=True)

    route_type='YAC'
    y = yac_routes[:,0]
    x = yac_routes[:,1]
    y_loc = (LOS - y)/_1_yd_y
    x_loc = (x - center_x)/_1_yd_x
    loc = [x_loc,y_loc]
    t_loc = np.transpose(loc)
    df = pd.DataFrame(t_loc,columns=['x','y'])
    df['route_type'] = route_type
    route_locations = route_locations.append(df, ignore_index=True)

    route_type='INCOMPLETE'
    y = inc_routes[:,0]
    x = inc_routes[:,1]
    y_loc = (LOS - y)/_1_yd_y
    x_loc = (x - center_x)/_1_yd_x
    loc = [x_loc,y_loc]
    t_loc = np.transpose(loc)
    df = pd.DataFrame(t_loc,columns=['x','y'])
    df['route_type'] = route_type
    route_locations = route_locations.append(df, ignore_index=True)
    
    return route_locations

In [ ]:
%%time
#Translate the image data into coordinate data for each chart ('main.py')
def get_route_data(data_file): 
	"""
	Extract the number of complete passes, incomplete passes, touchdowns, and interceptions 
	from a pass chart. 
	"""
	with open(data_file) as _file: 
		data = json.load(_file)
		_file.close()
	n_receptions = data["receptions"]
	n_touchdowns = data["touchdowns"]

	return (n_receptions, n_touchdowns)

def get_image(folder, data_file):
	"""
	If cleaned image exists, return the file path to the image, 
	otherwise return none.
	"""
	images_path = os.sep.join(folder.split(os.sep)[:-1]) + os.sep + "images" 
	image_file = images_path + os.sep + data_file.split(".")[0] + ".jpeg"
	if not os.path.exists(image_file):
		return None
	else:
		return image_file

def get_game_data(data_file):
	"""
	Extract player name, team, and game ID from the data corresponding to a pass chart image.
	"""	
	with open(data_file) as _file: 
		data = json.load(_file)
		_file.close()
	name = data["firstName"] + " " + data["lastName"]
	team = data["team"]
	game_id = data["gameId"]
	position = data["position"]
	week = data_file.split(os.sep)[-3]
	return (name, team, game_id, week, position)

def write_route_locations(image, data, routes):
	"""
	Write player, team, and game information, and locations of all passes to a .csv file.
	"""
	(n_route, n_td) = get_route_data(data)
	(name,team, game_id, week, position) = get_game_data(data)
	n_total = n_route

	route_cols = ["route_type", "x", "y"]
	game_cols = ["game_id", "team", "week", "name", 'position']
	route_df = pd.DataFrame(columns = route_cols)

	if (image is None): 
		rows_route = pd.DataFrame([["NAN", None, None]]*n_route, 
			columns = route_cols)
		route_df = route_df.append([rows_route])
		game_df = pd.DataFrame([[game_id, team, week, name, position]]*route_df.shape[0], 
			columns = game_cols)
		df = pd.concat([game_df, route_df.reset_index(drop=True)], axis=1)
		routes = routes.append(df)
		return routes

	if n_total != 0: 
		rows_route = map_route_locations(image,n_td)
		route_df = route_df.append(rows_route)

	game_df = pd.DataFrame([[game_id, team, week, name, position]]*route_df.shape[0], 
		columns = game_cols)

	df = pd.concat([game_df, route_df.reset_index(drop=True)], axis=1)
	routes = routes.append(df)
	return routes


if __name__ == '__main__':

	clean_path = "Cleaned_Route_Charts"
	routes = pd.DataFrame(columns = ["game_id", "team", "week", "name", "position", "route_type", "x", "y"])

	route_chart_folders = [folder[0] for folder in os.walk(clean_path)]
	data_folders = [folder for folder in route_chart_folders if folder.split(os.sep)[-1] == "data"]

	# td.to_csv('td_points.csv',index=False)
	td = pd.read_csv('td_points.csv')
	td_points = np.array(td[['x_coord','y_coord']])
    
	print("Extracting route locations...")
	for folder in data_folders:
		if ('2017' in folder):
# 			print(folder)
			data = os.listdir(folder)
			print(folder)
			for data_file in data:
				if not data_file.startswith("."): 
					print(data_file)
					image = get_image(folder, data_file)
					routes = write_route_locations(image, os.path.join(folder, data_file), routes)
	routes.to_csv("route_locations_sansTD_2017.csv", index=False, header=True)
	print("Done.")